### 링크
https://suwani.tistory.com/160

In [ ]:
!pip torch torchvision torchaudio

In [ ]:
import torch

print(torch.backends.mps.is_available())

True


In [ ]:
# 필수 라이브러리 설치
!pip install -U transformer datasets accelerate evaluate sentencepiece

# transformer : 허깅페이스 모델 및 Trainer
# datasets : 허깅페이스 데이터셋 로딩
# accelerate : GPU/CPU 자동 최적화
# evaluate : 정확도, F1 등 계산
# sentencepiece : 한국어 모델용 토크나이저 지원

## 데이터셋 로딩 + 전처리

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

dataset = load_dataset("glue", "sst2") # 감성분석
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(batch) :
    return tokenizer(batch["sentence",]
emotions = load_dataset("emotion")
emotions["train"].features["label"]
# 출력: ClassLabel(num_classes=6, names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)

/opt/anaconda3/envs/tfm/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating test split: 100%|██████████| 2000/2000 [00:00<00:00, 746782.52 examples/s]


ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'])

## 사전 훈련된 모델 사용

AutoModel 클래스에서 사전 훈련된 모델의 가중치 로드 -> from_pretrained() 메서드 사용

In [6]:
from transformers import AutoModel
import torch

model_ckpt = "distilbert-base-uncased"
use_mps = torch.backends.mps.is_available() and torch.backends.mps.is_built()
device = "mps" if use_mps else ("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

사전 모델 사용 순서 
1. Tokenizer 불러오기
2. 문자열 인코딩  
3. 모델에 입력해서 은닉 상태 반환 


In [ ]:
# 1. tokenizer 불러오기
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

# 2. 문자열 인코딩
text = "this is a text"
inputs = tokenizer(text, return_tensors="pt")
print(f"입력 텐서 크기 : {inputs['input_ids'].size()}")

# 3. 모델에 입력해서 은닉상태 반환
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
    outputs = model(**inputs)
print(f"outputs: {outputs}")
print(f"last hidden state: {outputs.last_hidden_state.size()}")

입력 텐서 크기 : torch.Size([1, 6])
outputs: BaseModelOutput(last_hidden_state=tensor([[[-0.2535, -0.1091,  0.0759,  ..., -0.1054,  0.2257,  0.4615],
         [-0.7154, -0.2742, -0.0954,  ..., -0.3597,  0.5174,  0.3450],
         [-0.4100, -0.2195,  0.1850,  ..., -0.0136,  0.1607,  1.0422],
         [-0.2523, -0.0254,  0.2472,  ..., -0.1637,  0.0865,  1.0516],
         [-0.0570, -0.1057,  0.0662,  ...,  0.0584,  0.1034,  0.2971],
         [ 0.9184,  0.1577, -0.4105,  ...,  0.1509, -0.7327, -0.1572]]],
       device='mps:0'), hidden_states=None, attentions=None)
last hidden state: torch.Size([1, 6, 768])
